In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql

In [2]:
connection = psycopg2.connect(
    host="localhost",
    database="grocery_db", 
    user="postgres",        
    password="1234" 
)

In [3]:
cursor = connection.cursor()
tables = ['grocery_logs', 'student_demographics', 'consumption_patterns', 'events', 'inventory_details', 'weather_data']

data_dict = {}

for table in tables:
    cursor.execute(f"SELECT * FROM {table}")
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]  
    data_dict[table] = pd.DataFrame(data, columns=columns)

for table, df in data_dict.items():
    print(f"Data from {table}:")
    print(df.head()) 
    print("\n")

cursor.close()
connection.close()


Data from grocery_logs:
   id item_name  quantity date_purchased    cost    supplier
0   1      rice       100     2024-01-01  200.00  Supplier A
1   2      milk        50     2024-01-01  100.00  Supplier B
2   3     Bread       120     2024-10-01  150.00  Supplier C
3   4      Eggs       200     2024-10-02  100.00  Supplier A
4   5    Butter        80     2024-10-03  160.00  Supplier D


Data from student_demographics:
   student_id  home_state dietary_preference  age gender allergies
0           1  Tamil Nadu         vegetarian   20      M      None
1           2      Kerala     non-vegetarian   22      F   peanuts
2           3  Tamil Nadu         Vegetarian   20      M      None
3           4      Kerala     Non-Vegetarian   22      F   Peanuts
4           5   Karnataka         Vegetarian   23      M      None


Data from consumption_patterns:
         date item_name  quantity_consumed  remaining_stock
0  2024-01-01      Rice                 80               20
1  2024-01-01      M

In [4]:
file_paths = {
    "grocery_logs": "C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/grocery_logs.csv",
    "student_demographics": "C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/student_demographics.csv",
    "events": "C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/events.csv",
    "consumption_patterns": "C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/consumption_patterns.csv",
    "inventory_details": "C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/inventory_details.csv",
    "weather_data": "C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/weather_data.csv"
}
for table, df in data_dict.items():
    if table in file_paths:
        df.to_csv(file_paths[table], index=False)
        print(f"Data from {table} saved as CSV at {file_paths[table]}.")


Data from grocery_logs saved as CSV at C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/grocery_logs.csv.
Data from student_demographics saved as CSV at C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/student_demographics.csv.
Data from consumption_patterns saved as CSV at C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/consumption_patterns.csv.
Data from events saved as CSV at C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/events.csv.
Data from inventory_details saved as CSV at C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/inventory_details.csv.
Data from weather_data saved as CSV at C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/dataset/weather_data.csv.


In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
def clean_data(df):
    for column in df.columns:
        if df[column].dtype in ['float64', 'int64']:
            df[column].fillna(df[column].mean(), inplace=True)
        elif df[column].dtype == 'object':
            df[column].fillna("Unknown", inplace=True)
    df.drop_duplicates(inplace=True)
    for column in df.columns:
        if 'date' in column.lower():
            df[column] = pd.to_datetime(df[column], errors='coerce')
    return df

def enrich_data(df, table_name):
    if table_name == 'grocery_logs':
        df['total_cost'] = df['quantity'] * df['cost']
        if 'date_purchased' in df.columns:
            df['weekday'] = df['date_purchased'].dt.day_name()
    
    elif table_name == 'student_demographics':
        if 'age' in df.columns:
            df['age_group'] = pd.cut(df['age'], bins=[0, 18, 35, 50, 100], labels=['Young', 'Adult', 'Middle Age', 'Senior'])
    
    elif table_name == 'consumption_patterns':
        df['weekly_consumption'] = df['quantity_consumed'] * 7
    
    elif table_name == 'events':
        if 'start_date' in df.columns:
            df['event_month'] = df['start_date'].dt.month_name()
    
    elif table_name == 'inventory_details':
        if 'current_stock' in df.columns and 'reorder_level' in df.columns:
            df['needs_reorder'] = df['current_stock'] < df['reorder_level']
    
    elif table_name == 'weather_data':
        if 'temperature' in df.columns:
            df['temperature_celsius'] = (df['temperature'] - 32) * 5.0/9.0
    
    return df

def standardize_data(df):
    scaler = StandardScaler()
    minmax_scaler = MinMaxScaler()
    
    for column in df.select_dtypes(include=[np.number]).columns:
        df[f"{column}_standardized"] = scaler.fit_transform(df[[column]])
        df[f"{column}_scaled"] = minmax_scaler.fit_transform(df[[column]])
    
    return df

def aggregate_data(df, table_name):
    if table_name == 'grocery_logs':
        return df.groupby('item_name').agg({
            'quantity': 'sum',
            'total_cost': 'sum'
        }).reset_index()
    
    elif table_name == 'consumption_patterns':
        return df.groupby('item_name').agg({
            'quantity_consumed': 'sum',
            'weekly_consumption': 'sum'
        }).reset_index()
    
    elif table_name == 'inventory_details':
        return df.groupby('item_name').agg({
            'current_stock': 'sum'
        }).reset_index()
    
    return pd.DataFrame()
transformed_file_path = "C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/transformed/"
aggregated_file_path = "C:/Users/Sweety Chittineni/Downloads/grocery recommendation system/aggregated/"
ensure_directory_exists(transformed_file_path)
ensure_directory_exists(aggregated_file_path)

transformed_data = {}
for table_name, df in data_dict.items():
    print(f"Processing table: {table_name}")
    df = clean_data(df)
    df = enrich_data(df, table_name)
    df = standardize_data(df)
    df_aggregated = aggregate_data(df, table_name)
    transformed_data[table_name] = df
    transformed_data[f"{table_name}_aggregated"] = df_aggregated
    df.to_csv(f"{transformed_file_path}{table_name}_transformed.csv", index=False)
    if not df_aggregated.empty:
        df_aggregated.to_csv(f"{aggregated_file_path}{table_name}_aggregated.csv", index=False)
    
    print(f"Transformed and saved data for {table_name}.")

print("ETL process completed.")


Processing table: grocery_logs
Transformed and saved data for grocery_logs.
Processing table: student_demographics
Transformed and saved data for student_demographics.
Processing table: consumption_patterns
Transformed and saved data for consumption_patterns.
Processing table: events
Transformed and saved data for events.
Processing table: inventory_details
Transformed and saved data for inventory_details.
Processing table: weather_data
Transformed and saved data for weather_data.
ETL process completed.
